# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [9]:
import os
import pandas as pd
import re
import csv

In [10]:
csv.field_size_limit(10000000) 
eval_path = '/Users/ulugsali/Desktop/Cognitive-Distortion-Project/processed_dataset.csv'
inference_df = pd.read_csv(eval_path)

pd.set_option('display.max_colwidth', None)
inference_df.head()

,Unnamed: 0,Prompt,Response,Class_1,Class_2
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.","Let's start eliminating the least likely classes present in the anecdote:\n\n1. Fortune-telling: This class is unlikely because there is no indication that the patient expects a specific outcome or believes that an event will always result in the worst possible outcome.\n\n2. Mind Reading: While the patient does mention their husband's behavior, there is no evidence to suggest that they believe he is intentionally trying to make them feel bad or react negatively without basis.\n\n3. Personalization: The patient does take responsibility for some of their actions and decisions, but there is also a significant amount of blame placed on their husband's behavior, making this class less likely.\n\n4. Labeling: There is no explicit labeling of themselves or their husband as ""loser"" or any other negative label, making this class less likely.\n\n5. Should Statements: While the patient does express some expectations about what they want from their relationship (e.g., wanting him to be more like he used to be), these statements are not overly rigid or absolute, making this class less likely.\n\n6. All-or-Nothing Thinking: The patient's descriptions of their husband's behavior do not suggest a binary thinking pattern where anything short of perfection is seen as a failure.\n\n7. Emotional Reasoning: While the patient does express strong emotions and frustration, there is no indication that they believe their feelings are objective truth or evidence of reality.\n\n8. Magnification: The patient does exaggerate some aspects of their husband's behavior (e.g., saying he constantly critiques them), but this class seems less likely because it doesn't capture the full scope of their concerns.\n\n9. Overgenerali

Fix Common Variations

In [ ]:
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return re.sub(r'\n', '', text)

inference_df['Response'] = inference_df['Response'].apply(lambda x: x.lower())  
inference_df['Response'] = inference_df['Response'].apply(remove_punctuation)  

inference_df.head()

AttributeError: 'float' object has no attribute 'lower'

Create Mapping Function

In [ ]:
distortions = {
    'none': 0,
    'no distortion': 0,
    'emotional reasoning': 1,
    'overgeneralization': 2,
    'mental filter': 3,
    'should statements': 4,
    'all-or-nothing thinking': 5,
    'mind reading': 6,
    'cognitive distortion: fortune-telling': 7,
    'magnification': 8,
    'minimization': 8,
    'magnification/minimization': 8,
    'personalization': 9,
    'labeling': 10,
}

patterns = {
    'none': r'\bnone\b',
    'no distortion': r'\bno distortion\b',
    'no distortion': r'no distortion',
    'emotional reasoning': r'\bemotional reasoning\b',
    'overgeneralization': r'\bovergeneralization\b',
    'mental filter': r'\bmental filter\b',
    'should statements': r'\bshould statements\b',
    'all-or-nothing thinking': r'\ball-or-nothing thinking\b',
    'all-or-nothing thinking': r'\ball or nothing thinking\b',
    'mind reading': r'\bmind reading\b',
    'magnification/minimization': r'\bmagnification/minimization\b',
    'magnification': r'\bmagnification\b',
    'minimization': r'\bminimization\b',
    'personalization': r'\bpersonalization\b',
    'labeling': r'\blabeling\b',
}

def catch_responses(x):
    # Keep hyphenation intact but lowercase everything
    normalized_text = x.lower()
    for label, pattern in patterns.items():
        match = re.search(pattern, normalized_text)
        if match:
            return distortions[label]
    return -1

Map common desired input, display any that are undesired for fine handling... For me, any denial should be thrown away (by being labelled -1).

In [ ]:
inference_df['Response'] = inference_df['Response'].apply(catch_responses)
inference_df.head(20)

,Unnamed: 0,Prompt,Response
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.",3
1,1,"I used to get many strange looks for the things I did, like smash a bird on the beach with a rock when I was younger, blow fish up with fireworks, laugh when a rock was thrown at a bird and broke its wing, and I also used to abuse pets a lot as well. I used to beat the dog, I would choke my pet hamster until his eyes started bugging out, stuff like that.",10
2,2,"Moved to another state left everything for my son who moved here bf me. He lives with dad and stepmom. Been a nightmare since I moved here. Now I have no job, no money, no family support. My friend back home said he’ll get me home, he sends money and not a plane or bus ticket. I’m so sad and hurt, haven’t caused issues for the ex and his new wife, just wanted to be here for my son’s senior year. I cry everyday! (age 44, from US)",9
3,3,"It has been more than a year now , I feel alone, depressed, don’t feel like talking to anyone, just like to be alone. Also have been too much emotional these days. I don’t have any girl friend or even a person who cares for me , except my mother, in my entire life. This all has started because I cant see my mother not getting respect from my father after staying with him for almost 30 years. I feel this so much , but I am not able to speak on this with either of them. I just keep on crying from inside. I cry while sleeping thinking of all this. This is affecting my behavior and I am no more a social person. I have lost all my friends. Even in office I just talk work. Some of my close friends say that I inspire them with the way I work, but when I come back home I just see all those things which I have seen over the years which makes me to forget the kind of person I am or I was to be. I have never got love. Beca

Gather Gold Data

In [ ]:
gold_dominant_data = pd.read_csv('../../datasets/test.csv')['Dominant Distortion']\
    .apply(lambda x: x.lower())\
    .apply(catch_responses)
gold_secondary_data = pd.read_csv('~/Cognitive-Distortion-Project/datasets/test.csv')['Secondary Distortion (Optional)']\
    .apply(lambda x: x if x is None else str(x).lower())\
    .apply(catch_responses)

inference_df['gold_dominant'] = gold_dominant_data
inference_df['gold_secondary'] = gold_secondary_data

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ulugsali/Cognitive-Distortion-Project/datasets/test.csv'

In [ ]:
def match_either(row):
    if row['Response'] == -1:
        row['gold'] = row['gold_dominant']
        return row
    if row['Response'] == row['gold_secondary']:
        row['gold'] = row['gold_secondary']
    else:
        row['gold'] = row['gold_dominant']
    return row

In [ ]:
inference_df = inference_df.apply(match_either, axis=1)

In [ ]:
print("Unique classes in 'gold':", inference_df['gold'].unique())
print("Unique classes in 'Response':", inference_df['Response'].unique())
#num in each class
print(inference_df['gold'].value_counts())

Unique classes in 'gold': [ 2  0  3  6  1 10  4 -1  9  8]
Unique classes in 'Response': [ 3 10  9  6  0 -1  4  1  8  2]
gold
 0     163
 2      59
 6      53
-1      52
 10     41
 8      35
 3      28
 9      28
 1      25
 4      22
Name: count, dtype: int64


Compute Accuracy and F1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
f1_macro = f1_score(inference_df['gold'], inference_df['Response'], average=None)
f1_scores_per_class = f1_score(inference_df['gold'], inference_df['Response'], labels=range(0, 11), average=None)
print("F1 scores for each class:")
for i, f1 in enumerate(f1_scores_per_class):
    print(f"{f1}")
print()
print(f1_score(inference_df['gold'], inference_df['Response'], labels=range(0,11), average='macro'))
print(accuracy_score(inference_df['gold'], inference_df['Response']))
print(f"F1-Score (Weighted): {f1_score(inference_df['gold'], inference_df['Response'], average='weighted')}")

Evaluated Model: multiclass Prompt: gemma-defined
F1 scores for each class:
0.4031620553359684
0.14285714285714285
0.16216216216216217
0.14084507042253522
0.13636363636363635
0.0
0.2777777777777778
0.0
0.0
0.16091954022988506
0.30952380952380953

0.15760101769753795
0.2391304347826087
F1-Score (Weighted): 0.25361415724838804


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
print(f"Accuracy: {accuracy_score(inference_df['gold_dominant'], inference_df['Response'])}")
f1_macro = f1_score(inference_df['gold_dominant'], inference_df['Response'], labels=[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], average=None)
for i, f1 in enumerate(f1_macro):
    print(f"F1-Score (Macro, Class {i}): {f1}")
print(f"F1-Score (Macro): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='macro')}")
print(f"F1-Score (Weighted): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='weighted')}")

Evaluated Model: multiclass Prompt: gemma-defined
Accuracy: 0.22134387351778656
F1-Score (Macro, Class 0): 0.19801980198019803
F1-Score (Macro, Class 1): 0.4031620553359684
F1-Score (Macro, Class 2): 0.12612612612612611
F1-Score (Macro, Class 3): 0.13333333333333333
F1-Score (Macro, Class 4): 0.11428571428571428
F1-Score (Macro, Class 5): 0.11494252873563218
F1-Score (Macro, Class 6): 0.0
F1-Score (Macro, Class 7): 0.24528301886792453
F1-Score (Macro, Class 8): 0.0
F1-Score (Macro, Class 9): 0.0
F1-Score (Macro, Class 10): 0.11764705882352941
F1-Score (Macro, Class 11): 0.2823529411764706
F1-Score (Macro): 0.17351525786648972
F1-Score (Weighted): 0.23826110586156327


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#print where response == -1
inference_df[inference_df['Response'] == -1]

,Unnamed: 0,Prompt,Response,gold_dominant,gold_secondary,gold
10,10,"I’ve been having emotional issues for a few years. Nonetheless, these have been somewhat manageable. However, I became increasingly paranoid this winter. I thought that people were trying to poison me, I feared that family members were going to kill me. I was aware that these thoughts were illogical, but I couldn’t shake the fear that they caused me. I would get panic attacks thinking that I drank from a poisoned water fountain. I thought that someone had put small rips in my ice cream sandwich packaging because there was only one that didn’t have tiny rips in it. I refused to eat this one because I assumed some reverse psychology was being pulled and the ones without the rips were fine. I thought that this was mostly caused by anxiety but there are other symptoms that lead me to believe it is something more:",-1,1,-1,1
18,18,"I don’t know what’s wrong with me, and its driving me crazy. I was dx with schizophrenia, but even though I feel like I don’t have that, just knowing about it doesn’t help anything at all. I am in an early psychosis program, and they’ve helped me a lot but I still feel like I’m not getting any better with some things. I feel like I can never be normal or live normally….",-1,0,-1,0
47,47,"I’ve been suffering from some issues since I was little. I have always been scared when it’s dark. I hear noises in the house and outside, but I don’t see what’s making the noises. It makes me feel afraid to go to sleep and I usually end up being awake until the sun rises. If I do fall asleep, I almost always get awakened by nightmares. These nightmares often involve someone breaking in, violence, someone haunting me (usually children who went missing/were killed or something that was on TV), abandonment, or being trapped.",-1,1,-1,1
54,54,"I have been struggling with OCD and anxiety for the past 6 months. It has been an intense 6 months and I am still struggling daily although I am trying my best to do the little things that help me to not be so obsessive over my worries. Recently, I have been worried about one day going insane and not being able to control it. I once watched a movie about a man who was able to time travel against his own will, everytime he woke up from his sleep. I know this is ridiculous but I actually thought what if that happens to me? And feared for a couple days straight. I know this is impossible and now that I look back on it, I realize that this is almost laughable. Although I no longer fear that I will be able to time travel, I’m afraid of the fact that I even feared such a ridiculous, impossible scenario. It triggered anxiety in me because I thought that I was going crazy. It was different from any other fear that I had because this was a fear that was sprouted from fiction while all of my other fears sprouted from things that happened in real life. (Ex. Cancer). I researched online and read that people who are crazy have a hard time differentiating reality from fiction. But during those two day of fear, it was almost as if It could happen to me (which I know that it couldn’t). Am I going crazy? I’m really afraid. Thank you for your response.",-1,-1,-1,-1
59,59,"Hey i am Ay. from Turkey Istanbul. I will get straight to the point. I used to scare death but this summer i started to not. I dont know why but I actually started to want to be dead. But i was scared that what if i die and nothing gonna happen like all empytness and black screen but sometimes i thought myself maybe i will go heaven and there is god. I searched the internet and i found out that I have anxiety but i dont believe that. I dont know why but i feel like i am going to die. No, i am not going to suicide and never tried that but i kinda feel like it is going to be a traffic accident or someone will kill me. And now i dont fear nothing . I sometimes think of my funeral and ask myself how many people will attempt? I always get sad that my family will cry a lot